In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
ratings = pd.read_csv(r"C:\Users\sivar\Downloads\ml-25m\ml-25m\ratings.csv", sep=',', encoding='latin-1',usecols=['userId','movieId','rating'])
users = pd.read_csv(r"C:\Users\sivar\Downloads\ml-25m\ml-25m\tags.csv", sep=',', encoding='latin-1', usecols=['userId','movieId','tag'])
movies = pd.read_csv(r"C:\Users\sivar\Downloads\ml-25m\ml-25m\movies.csv", sep=',',encoding='latin-1', usecols=['movieId' , 'title' ,'genres'])

genre_labels = set()
for s in movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

# Function that counts the number of times each of the genre keywords appear
def count_word(dataset, ref_col, census):
    keyword_count = dict()
    for s in census: 
        keyword_count[s] = 0
    for census_keywords in dataset[ref_col].str.split('|'):        
        if type(census_keywords) == float and pd.isnull(census_keywords): 
            continue        
        for s in [s for s in census_keywords if s in census]: 
            if pd.notnull(s): 
                keyword_count[s] += 1
    #______________________________________________________________________
    # converting the dictionary in a list to sort the keywords by frequency
    keyword_occurences = []
    for k,v in keyword_count.items():
        keyword_occurences.append([k,v])
    keyword_occurences.sort(key = lambda x:x[1], reverse = True)
    return keyword_occurences, keyword_count
# Calling this function gives access to a list of genre keywords which are sorted by decreasing frequency
keyword_occurences, dum = count_word(movies, 'genres', genre_labels)
keyword_occurences[:5]

[['Drama', 25606],
 ['Comedy', 16870],
 ['Thriller', 8654],
 ['Romance', 7719],
 ['Action', 7348]]

In [13]:
# Breaking up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Converting genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')


from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix
 


from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]


# Building a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


 genre_recommendations('movie name').head(10) # enter the movie name here to get recommendation

MemoryError: Unable to allocate 4.46 GiB for an array with shape (1197641999,) and data type int32